In [3]:
print("ok")

ok


In [4]:
import os
os.chdir('../')

In [5]:
%pwd

'd:\\CodeCommand\\End to End Medical Chatbot Project Implementation  Generative AI  English  Euron'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [7]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_file(data='Data/')

incorrect startxref pointer(1)
parsing for Object Streams


In [9]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=20
    )
    texts = text_splitter.split_documents(documents=extracted_data)
    return texts

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 557


In [11]:
# text_chunks

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings= download_hugging_face_embeddings()

C:\Users\Nazbeen Ai\AppData\Local\Temp\ipykernel_13588\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [30]:
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY=os.environ.get(key='PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get(key='GROQ_API_KEY')

In [ ]:
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'medicalbot'

pc.create_index(
  name=index_name,
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)


{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-m67xe5j.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY


In [18]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [19]:
#  Load Existing index

docsearch = PineconeVectorStore.from_existing_index(
    index_name,
    embedding=embeddings
    )
docsearch

In [20]:
retriever = docsearch.as_retriever(
    search_type='similarity',
    search_kwargs={"k": 5}
)

In [23]:
retriever_docs = retriever.invoke('what is DeeperGoogLeNet on Tiny ImageNet?')

In [24]:
retriever_docs

[Document(id='17f2d0f8-cbbe-4125-a3fd-eb63bbfb66f0', metadata={'author': 'Adrian Rosebrock', 'creationdate': '2017-11-29T09:00:19-05:00', 'creator': 'calibre (3.39.1) [https://calibre-ebook.com]', 'keywords': '00 Важно', 'moddate': '2019-11-02T16:27:44+03:00', 'page': 8.0, 'page_label': '7', 'producer': 'pdfTeX-1.40.16', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.16 (TeX Live 2015) kpathsea version 6.2.1', 'source': 'Data\\deep-learning-for-computer-vision-with-python-starter-bundle_compress.pdf', 'subject': '', 'title': 'Deep Learning for Computer Vision with Python. 2 - Practitioner Bundle', 'total_pages': 210.0, 'trapped': '/False'}, page_content='11.2.5 MiniGoogLeNet: Experiment #3 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 145\n11.3 The Tiny ImageNet Challenge 146\n11.3.1 Downloading Tiny ImageNet . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 147\n11.3.2 The Tiny ImageNet Directory Structure . . . . . 

In [32]:

from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.4,
    max_tokens=500
)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = """
You are an intelligent AI assistant designed to provide accurate and relevant responses based on retrieved knowledge. Your primary goal is to assist users by leveraging the available information effectively.

Guidelines:
- If the retrieved documents contain relevant information, provide a well-structured and concise answer.
- If the retrieved data is insufficient or unavailable, respond professionally:
  **"I'm sorry, but I don't have enough information to answer that."**
- Maintain clarity, professionalism, and a helpful tone in all responses.
- Avoid making assumptions or generating misleading information.
- When necessary, encourage users to refine their query for better results.

Your responses should always be structured, informative, and aligned with the given context.

Context:
{context}
"""


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)


In [35]:
question_answer_chain =  create_stuff_documents_chain(llm, prompt_template)
rag_chain =  create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
import pprint


response = rag_chain.invoke({
    "input": "What is Fine-tuning Networks?"
})
pprint.pp(response['answer'])

('Fine-tuning is a type of transfer learning that involves modifying and '
 're-training parts of a deep learning model that has already been trained on '
 'a specific dataset. Typically, these networks are state-of-the-art '
 'architectures such as VGG, ResNet, and Inception, which have been trained on '
 'the ImageNet dataset.\n'
 '\n'
 'During fine-tuning, the final set of layers (the "head") is removed from the '
 'network, and the body of the original network is frozen while new '
 'fully-connected (FC) layers are trained. Once the network starts to obtain '
 'reasonable accuracy, part (if not all) of the body is unfrozen, allowing '
 'training to continue. This technique leverages pre-existing network '
 'architectures and their rich, discriminative set of filters, enabling faster '
 'learning and higher accuracy transfer learning models with less effort than '
 'training from scratch.\n'
 '\n'
 'Before performing network surgery, it is essential to know the layer names '
 'and i